# 역전파 (Backpropagation)

### 연쇄법칙

- 기본 수식의 역전파 & 연쇄법칙 적용

In [ ]:
import numpy as np

def forward(x):
    y = x ** 2
    return y

def backward(x):
    dy_dx = 2 * x
    return dy_dx

x = 3.0     # x : 출력값에 대한 손실
print(forward(x))
print(backward(x))

9.0
6.0


- 다층 신경망에서 연쇄법칙 적용

In [3]:
def forward(x):
    y = x ** 2
    z = 2 * y
    return z

def backward(x):
    dy_dx = 2 * x
    dz_dy = 2
    dz_dx = dz_dy * dy_dx 
    return dz_dx

x = 3.0
print(forward(x))
print(backward(x))

18.0
12.0


### 신경망에서의 활용

- 단순 신경망 학습

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_d(x):
    return sigmoid(x) * (1 - sigmoid(x))

X = np.array([0.5, 0.8])
y = np.array([1])

W = np.array([0.2, 0.4])

# 순전파
z = np.dot(X, W)
r = sigmoid(z)

# 오차 계산
loss = 0.5 * (y - r) ** 2   # mse에 계산을 간단히 해주기 위한 상수 0.5를 곱해준 것을 loss로 잡는다 (mse의 변형) 

# 역전파 (기울기 계산)
delta = (r - y) * sigmoid_d(z)  # r에 대해 z가 미치는 영향  / 출력층에서의 오차 / chain rule (d_loss_dz = d_loss_dr * dr_dz)
grad_w = delta * X          # z = np.dot(X, W) -> d_loss_d_W = d_loss_dz * dz_dW / 여기서 dz_dW = X, d_loss_dz = delta => d_loss_dW(grad_w) = delta * X

# 가중치 갱신
W -= 0.1 * grad_w   # 0.1 : learning_rate   /  기울기를 이용한 가중치 업데이트 

print(W)

[0.20474415 0.40759064]


- 은닉층 추가

In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_d(x):
    return np.where(x > 0, 1, 0)

X = np.array([0.5, 0.8])    # (2,)
y = np.array([1])

W1 = np.array([[0.2, 0.4], [0.1, 0.3]]) # (2, 2)
b1 = np.array([0.1, 0.2])   # (2,)
W2 = np.array([[0.5], [0.6]])   # (2, 1)
b2 = np.array([0.3])    

# 순전파
z1 = np.dot(X, W1) + b1
r1 = relu(z1)

z2 = np.dot(r1, W2) + b2
r2 = relu(z2)

# 역전파 (기울기 계산)  # 손실함수 = 1/2 * (예측값 - 실제값) ^2 -> 미분 -> (예측값 - 실제값)*(활성화함수 미분) : (r2 - y)*relu_d(z2)
delta2 = (r2 - y) * relu_d(z2)  # 오차 / relu_d(층에 대한 input값)
grad_W2 = np.outer(r1, delta2)

delta1 = np.dot(W2, delta2) * relu_d(z1)
grad_W1 = np.outer(X, delta1)

# 가중치 갱신 
learning_rate = 0.01
W2 -= learning_rate * grad_W2
W1 -= learning_rate * grad_W1

print(W2)
print(W1)



[[0.5004928]
 [0.6011264]]
[[0.20044   0.400528 ]
 [0.100704  0.3008448]]


### 수치미분과 역전파

In [ ]:
def f(x):
    return x ** 2

def num_d_gradient(f, x):
    h = 1e-5
    return (f(x + h) - f(x - h)) / (2 * h)  # 중앙차분법 수치미분 

def backward_gradient(x):   # f 미분 
    return 2 * x

# 비교 (기울기 검증) 
x = 3.0
print(num_d_gradient(f, x))
print(backward_gradient(x))

6.000000000039306
6.0


##### 숫자 맞추기 AI

In [18]:
target_number = 42

guess = np.random.randn()   # 초기 예측값. 예측값이 주어진 상태로 시작하므로 순전파는 없음 

learning_rate = 0.1

for i in range(500):
    # 오차 계산
    loss = 0.5 * (target_number - guess) ** 2

    # 역전파 (기울기 계산)
    grad = (guess - target_number)

    # 업데이트 (guess 업데이트)
    guess -= learning_rate * grad

    # epoch 50마다 예측값과 손실 출력
    if i % 50 == 0:
        print(f'epoch {i} | 예측값: {guess} , 손실: {loss}')

# 최종 예측값 guess 출력
print(f'최종 예측값: {guess}')

epoch 0 | 예측값: 3.2544197693172947 , 손실: 926.6790045754756
epoch 50 | 예측값: 41.80031398921388 , 손실: 0.024613890681280226
epoch 100 | 예측값: 41.99897086318836 , 손실: 6.53779368557667e-07
epoch 150 | 예측값: 41.99999469606021 , 손실: 1.7365294583741053e-11
epoch 200 | 예측값: 41.99999997266468 , 손실: 4.612466732472891e-16
epoch 250 | 예측값: 41.99999999985913 , 손실: 1.2250109698369697e-20
epoch 300 | 예측값: 41.99999999999927 , 손실: 3.280651623760726e-25
epoch 350 | 예측값: 41.99999999999997 , 손실: 4.0389678347315804e-28
epoch 400 | 예측값: 41.99999999999997 , 손실: 4.0389678347315804e-28
epoch 450 | 예측값: 41.99999999999997 , 손실: 4.0389678347315804e-28
최종 예측값: 41.99999999999997
